In [52]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import create_optimizer, AdamWeightDecay
from transformers import TFAutoModelForCausalLM
import tensorflow as tf

In [2]:
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Reusing dataset eli5 (C:\Users\SP003DA2\.cache\huggingface\datasets\eli5\LFQA_reddit\1.0.0\17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [3]:
eli5 = eli5.train_test_split(test_size=0.2)

In [21]:
eli5

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 1000
    })
})

In [4]:
eli5["train"][0]

{'q_id': 'o2or7',
 'title': "How does a snake's digestive system handle materials that range so widely in composition?\n",
 'selftext': "That is, how does it manage to break down another animal's body in its entirety?\n\nreddit post from which my question stems:\n_URL_0_\n\nEdit: Thanks for the feedback!",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['c3dxqtb', 'c3dvbug'],
  'text': ['[This news release](_URL_0_) explains that snake digestion has been speculation for a long time, but researchers now have evidence that the snake\'s intestine/liver/other digestive organs are dormant and "reduced" between meals.  Shortly after ingestion, they\'re activated and start producing mucosa, lengthening and increasing in diameter.  Digestion goes into high-gear, but everything else takes a backseat, with the snake becoming basically dormant as it digests.  Here\'s an exert from the above link:  \n\n\n > The diameter of the snake\'s intestines apparently increased by about 30

In [43]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

In [6]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': 'o2or7',
 'title': "How does a snake's digestive system handle materials that range so widely in composition?\n",
 'selftext': "That is, how does it manage to break down another animal's body in its entirety?\n\nreddit post from which my question stems:\n_URL_0_\n\nEdit: Thanks for the feedback!",
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['c3dxqtb', 'c3dvbug'],
 'answers.text': ['[This news release](_URL_0_) explains that snake digestion has been speculation for a long time, but researchers now have evidence that the snake\'s intestine/liver/other digestive organs are dormant and "reduced" between meals.  Shortly after ingestion, they\'re activated and start producing mucosa, lengthening and increasing in diameter.  Digestion goes into high-gear, but everything else takes a backseat, with the snake becoming basically dormant as it digests.  Here\'s an exert from the above link:  \n\n\n > The diameter of the snake\'s intestines apparently increased by about

In [7]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]], truncation=True)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    #num_proc=4, # This does not work on my environment. Maybe because no GPU ?
    remove_columns=eli5["train"].column_names,
)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
tokenized_eli5['train']

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4000
})

In [24]:
block_size = 128

In [25]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [27]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

In [47]:
tf_train_set = lm_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = lm_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [51]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [53]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [54]:
# Won't work: there's a problem with the size of the tensors when tokenizing
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
 27/526 [>.............................] - ETA: 1:38:32 - loss: 4.0838

InvalidArgumentError: Graph execution error:

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.
Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py", line 707, in convert_to_tensors
    tensor = as_tensor(value)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\constant_op.py", line 268, in constant
    allow_broadcast=True)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\constant_op.py", line 279, in _constant_impl
    return _constant_eager_impl(ctx, value, dtype, shape, verify_shape)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\constant_op.py", line 304, in _constant_eager_impl
    t = convert_to_eager_tensor(value, ctx, dtype)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\